In [2]:
import ee, datetime
import pandas as pd
import numpy as np
import datetime as dt

In [3]:
ee.Authenticate()
ee.Initialize()


Successfully saved authorization token.


In [4]:
geometry = ee.Geometry.Polygon([[[78.59718667470753,30.1451735459128]
 [78.59686480962574,30.144468428872372]
 [78.59669314824879,30.143355075916983]
 [78.59652148687184,30.14209326071362]
 [78.59650002919972,30.140942768082624]
 [78.59618267668829,30.14005394738466]
 [78.59617194785223,30.138810646811912]
 [78.59617194785223,30.138142598243498]
 [78.59677276267156,30.137864243338626]
 [78.5967083896552,30.13884776048867]
 [78.59685859336004,30.139107555835174]
 [78.59677276267156,30.139775597871388]
 [78.59694442404852,30.140963216991686]
 [78.59694442404852,30.141547743028735]
 [78.59707317008123,30.14269822860625]
 [78.59713754309759,30.142902345614043]
 [78.59706244124517,30.14319924232614]
 [78.59728774680242,30.14351469410388]
 [78.59724483145818,30.14390436843706]
 [78.59765252722845,30.144572377998944]
 [78.59851083411321,30.14514760483082]
 [78.59718667470753,30.1451735459128]]])

In [6]:
def get_turbidity(start_date, end_date):

# Selecting the satellite and AOI  
# Sentinel 2A
# copernicus/s2_sr 
    sentinel = ee.ImageCollection("COPERNICUS/S2_SR").\
               filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)).\
               filterDate(start_date, end_date)
    AOI = geometry

    sentinel_AOI = sentinel.filterBounds(AOI)

#calculate NDTI
    def calculate_NDTI(image):
        ndti = image.normalizedDifference(['B4', 'B3']).rename('NDTI')
        return image.addBands(ndti)
    ndti = sentinel_AOI.map(calculate_NDTI)

# Mean NDTI
    def calculate_mean_NDTI(image):
        image = ee.Image(image)
        mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['NDTI']),
                                geometry = AOI,
                                scale = image.projection().nominalScale().getInfo(),
                                maxPixels = 100000,
                                bestEffort = True);
        return mean.get('NDTI').getInfo()
    
# NDTI mean collection 
    Images_ndti = ndti.select('NDTI').toList(ndti.size())
    ndti_coll = []
    for i in range(Images_ndti.length().getInfo()):
        image = ee.Image(Images_ndti.get(i-1))
        temp_ndti = calculate_mean_NDTI(image)
        ndti_coll.append(temp_ndti)

# Dates Collection
    dates = np.array(ndti.aggregate_array("system:time_start").getInfo())
    day = [datetime.datetime.fromtimestamp(i/1000).strftime('%Y-%m-%d') for i in (dates)]

# Dataframe for Turtbidity

    df = pd.DataFrame(ndti_coll, index = day, columns = ['Turbidity'])
    df.index = pd.to_datetime(df.index, format="%Y/%m/%d")
    df.sort_index(ascending = True, inplace = True)

    return df

In [7]:
def get_Salanity(start_date, end_date):

# Selecting the satellite and AOI  
# Sentinel 2A
# copernicus/s2_sr 
    sentinel = ee.ImageCollection("COPERNICUS/S2_SR").\
               filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)).\
               filterDate(start_date, end_date)
    AOI = geometry

    sentinel_AOI = sentinel.filterBounds(AOI)

#calculate NDSI
    def calculate_NDSI(image):
        ndsi = image.normalizedDifference(['B11', 'B12']).rename('NDSI')
        return image.addBands(ndsi)
    ndsi = sentinel_AOI.map(calculate_NDSI)

# Mean NDSI
    def calculate_mean_NDSI(image):
        image = ee.Image(image)
        mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['NDSI']),
                                geometry = AOI,
                                scale = image.projection().nominalScale().getInfo(),
                                maxPixels = 100000,
                                bestEffort = True);
        return mean.get('NDSI').getInfo()
        
# NDSI Mean Collection
    Images_ndsi = ndsi.select('NDSI').toList(ndsi.size())
    ndsi_coll = []
    for i in range(Images_ndsi.length().getInfo()):
        image = ee.Image(Images_ndsi.get(i-1))
        temp_ndsi = calculate_mean_NDSI(image)
        ndsi_coll.append(temp_ndsi)

# Dates Collection
    dates = np.array(ndsi.aggregate_array("system:time_start").getInfo())
    day = [datetime.datetime.fromtimestamp(i/1000).strftime('%Y-%m-%d') for i in (dates)]

# Dataframe for Salinity

    df = pd.DataFrame(ndsi_coll, index = day, columns = ['Salinity'])
    df.index = pd.to_datetime(df.index, format="%Y/%m/%d")
    df.sort_index(ascending = True, inplace = True)

    return df

In [8]:
def get_DO(start_date, end_date):

# Selecting the satellite and AOI  
# Sentinel 2A
# copernicus/s2_sr 
    sentinel = ee.ImageCollection("COPERNICUS/S2_SR").\
               filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)).\
               filterDate(start_date, end_date)
    AOI = geometry

    sentinel_AOI = sentinel.filterBounds(AOI)

# calculate Dissolved Oxygen
    def calculate_DO(image):
        do = image.normalizedDifference(['B4', 'B5']).rename('DO')
        return image.addBands(do)
    do = sentinel_AOI.map(calculate_DO)

# Mean DO

    def calcualte_mean_DO(image):
        image = ee.Image(image)
        mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['DO']),
                            geometry = AOI,
                            scale = image.projection().nominalScale().getInfo(),
                            maxPixels = 100000,
                            bestEffort = True);
        return mean.get('DO').getInfo()
        
# DO Mean Collection

    Images_do = do.select('DO').toList(do.size())
    do_coll = []
    for i in range(Images_do.length().getInfo()):
        image = ee.Image(Images_do.get(i-1))
        temp_do = calcualte_mean_DO(image)
        do_coll.append(temp_do)

# Dates Collection
    dates = np.array(do.aggregate_array("system:time_start").getInfo())
    day = [datetime.datetime.fromtimestamp(i/1000).strftime('%Y-%m-%d') for i in (dates)]

# Dataframe for DO

    df = pd.DataFrame(do_coll, index = day, columns = ['Dissolved Oxygen'])
    df.index = pd.to_datetime(df.index, format="%Y/%m/%d")
    df.sort_index(ascending = True, inplace = True)

    return df

In [9]:
def get_pH(start_date, end_date):

# Selecting the satellite and AOI  
# Sentinel 2A
# copernicus/s2_sr 
    sentinel = ee.ImageCollection("COPERNICUS/S2_SR").\
               filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',20)).\
               filterDate(start_date, end_date)
    AOI = geometry

    sentinel_AOI = sentinel.filterBounds(AOI)

# calculate pH

    def calculate_pH(image):
        ph = ee.Image(8.339).subtract(ee.Image(0.827).multiply(image.select('B1').divide(image.select('B8')))).rename('PH')
        return image.addBands(ph)
    pH = sentinel_AOI.map(calculate_pH)

# Mean pH

    def calculate_mean_pH(image):
        image = ee.Image(image)
        mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['PH']),
                                geometry = AOI,
                                scale = image.projection().nominalScale().getInfo(),
                                maxPixels = 100000,
                                bestEffort = True);
        return mean.get('PH').getInfo()
        
# pH Mean Collection

    Images_ph = pH.select('PH').toList(pH.size())
    ph_coll= []
    for i in range(Images_ph.length().getInfo()):
        image = ee.Image(Images_ph.get(i-1))
        temp_ph = calculate_mean_pH(image)
        ph_coll.append(temp_ph)

# Dates Collection

    dates = np.array(pH.aggregate_array("system:time_start").getInfo())
    day = [datetime.datetime.fromtimestamp(i/1000).strftime('%Y-%m-%d') for i in (dates)]

# Dataframe for pH

    df = pd.DataFrame(ph_coll, index = day, columns = ['pH'])
    df.index = pd.to_datetime(df.index, format="%Y/%m/%d")
    df.sort_index(ascending = True, inplace = True)

    return df

In [11]:
def get_DOM(start_date, end_date):

# Selecting the satellite and AOI  
# Sentinel 3 OLCI
# Copernicus Sentinel-3 OLCI 
    sentinel3 = ee.ImageCollection("COPERNICUS/S3/OLCI").\
              filterDate(start_date, end_date)
    AOI = geometry

    sentinel3_AOI = sentinel3.filterBounds(AOI)

# calculate Dissolved Organic Matter (DOM) from Sentinel-3 OLCI

    def calculate_DM(image):
        rgb = image.select(['Oa08_radiance', 'Oa06_radiance', 'Oa04_radiance'])\
              .multiply(ee.Image([0.00876539, 0.0123538, 0.0115198]))
        dm = rgb.select('Oa08_radiance').divide(rgb.select('Oa04_radiance')).rename('dom')
        return image.addBands(dm)
    dm = sentinel3_AOI.map(calculate_DM)

# Mean DOM

    def calcualte_mean_DM(image):
        image = ee.Image(image)
        mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['dom']),
                            geometry = AOI,
                            scale = image.projection().nominalScale().getInfo(),
                            maxPixels = 100000,
                            bestEffort = True);
        return mean.get('dom').getInfo()
    
# DOM Mean Collection

    Images_dm = dm.select('dom').toList(dm.size())
    dm_coll= []
    for i in range(Images_dm.length().getInfo()):
        image = ee.Image(Images_dm.get(i-1))
        temp_dm = calcualte_mean_DM(image)
        dm_coll.append(temp_dm)

# Dates Collection

    dates = np.array(dm.aggregate_array("system:time_start").getInfo())
    day = [datetime.datetime.fromtimestamp(i/1000).strftime('%Y-%m-%d') for i in (dates)]

# Dataframe for DOM

    df = pd.DataFrame(dm_coll, index = day, columns = ['dom'])
    df.index = pd.to_datetime(df.index, format="%Y/%m/%d")
    df.sort_index(ascending = True, inplace = True)

    return df

In [12]:
def get_sm(start_date, end_date):

# Selecting the satellite and AOI  
# Sentinel 3 OLCI
# Copernicus Sentinel-3 OLCI 
    sentinel3 = ee.ImageCollection("COPERNICUS/S3/OLCI").\
              filterDate(start_date, end_date)
    AOI = geometry

    sentinel3_AOI = sentinel3.filterBounds(AOI)

# calculate suspended matter

    def calculate_SM(image):
        rgb = image.select(['Oa08_radiance', 'Oa06_radiance', 'Oa04_radiance'])\
              .multiply(ee.Image([0.00876539, 0.0123538, 0.0115198]))
        suspended_matter = rgb.select('Oa08_radiance').divide(rgb.select('Oa06_radiance')).rename('suspended_matter')
        return image.addBands(suspended_matter)
    sm = sentinel3_AOI.map(calculate_SM)

# Mean of Suspended Matter

    def meanSM(image):
        image = ee.Image(image)
        mean = image.reduceRegion(reducer = ee.Reducer.mean().setOutputs(['suspended_matter']),
                           geometry = AOI,
                           scale = image.projection().nominalScale().getInfo(),
                           maxPixels = 100000,
                           bestEffort = True);
        return mean.get('suspended_matter').getInfo()
            
# Suspended-Matter Mean Collection

    Images_sm = sm.select('suspended_matter').toList(sm.size())
    sm_coll= []
    for i in range(Images_sm.length().getInfo()):
        image = ee.Image(Images_sm.get(i-1))
        temp_sm = meanSM(image)
        sm_coll.append(temp_sm)

# Dates Collection

    dates = np.array(sm.aggregate_array("system:time_start").getInfo())
    day = [datetime.datetime.fromtimestamp(i/1000).strftime('%Y-%m-%d') for i in (dates)]

# Dataframe for suspended matter

    df = pd.DataFrame(sm_coll, index = day, columns = ['Suspended Matter'])
    df.index = pd.to_datetime(df.index, format="%Y/%m/%d")
    df.sort_index(ascending = True, inplace = True)

    return df

In [13]:
'''df1 = get_chlorophyll('2019-01-01', '2019-01-10')
   df1.to_csv('chlorophyll.csv')'''

#In this format, the data is stored in CSV.


"df1 = get_chlorophyll('2019-01-01', '2019-01-10')\n   df1.to_csv('chlorophyll.csv')"

In [15]:
turbidity = get_turbidity('2018-01-01', '2022-12-31')
temperature = get_temperature('2018-01-01', '2022-12-31')
DOM = get_DOM('2018-01-01', '2022-12-31')
suspended_matter = get_sm('2018-01-01', '2022-12-31')
pH = get_pH('2018-01-01', '2022-12-31')
dissolved_oxygen = get_DO('2018-01-01', '2022-12-31')

In [ ]:
turbidity.to_csv('turbidity_devprayag.csv', index=True)
temprature.to_csv('temprature_devprayag.csv', index=True)
pH.to_csv('ph_devprayag.csv', index=True)
DOM.to_csv('dom_devprayag.csv', index=True)
dissolved_oxygen.to_csv('dissolved_oxygen_devprayag.csv', index=True)
suspended_matter.to_csv('suspended_matter_devprayag.csv', index=True)
